# LOBSTER Workflow Tutorial (Using VASP for DFT)

The first lines are needed to ensure that we can mock VASP and LOBSTER runs. The test files here might not belong to the same calculation but are good enough for testing.

In [ ]:
import warnings
from mock_lobster import mock_lobster
from mock_vasp import TEST_DIR, mock_vasp

ref_paths = {
    "relax 1": "Si_lobster_uniform/relax_1",
    "relax 2": "Si_lobster_uniform/relax_2",
    "static": "Si_lobster_uniform/static",
    "non-scf uniform": "Si_lobster_uniform/non-scf_uniform",
}
ref_paths_lobster = {
    "lobster_run_0": "Si_lobster/lobster_0",
}

We first load a structure that we want to analyze with bonding analysis.

In [ ]:
from jobflow import JobStore, run_locally
from maggma.stores import MemoryStore
from pymatgen.core import Structure

from atomate2.vasp.flows.lobster import LobsterMaker, VaspLobsterMaker
from atomate2.vasp.powerups import update_user_incar_settings
warnings.filterwarnings('ignore')

job_store = JobStore(MemoryStore(), additional_stores={"data": MemoryStore()})
si_structure = Structure.from_file(TEST_DIR / "structures" / "Si.cif")

Then, we initialize a workflow:

In [ ]:
job = VaspLobsterMaker(
    lobster_maker=LobsterMaker(
        task_document_kwargs={
            "calc_quality_kwargs": {"potcar_symbols": ["Si"], "n_bins": 10},
            "add_coxxcar_to_task_document": True,
        },
        user_lobsterin_settings={
            "COHPstartEnergy": -5.0,
            "COHPEndEnergy": 5.0,
            "cohpGenerator": "from 0.1 to 3.0 orbitalwise",
        },
    ),
    delete_wavecars=True,
).make(si_structure)
job = update_user_incar_settings(job, {"NPAR": 4})

# run the flow or job and ensure that it finished running successfully

We then run this workflow locally to show-case the capabilities. In real-life, you would omit the `mock*` parts.

In [ ]:
with mock_vasp(ref_paths) as mf, mock_lobster(ref_paths_lobster) as mf2:
    responses = run_locally(
        job,
        store=job_store,
        create_folders=True,
        ensure_success=True,
        raise_immediately=True,
    )

We can also analyze the data from the database

In [ ]:
from pymatgen.electronic_structure.cohp import Cohp
from pymatgen.electronic_structure.plotter import CohpPlotter

store = job_store

result = store.query_one(
    {"name": "lobster_run_0"},
    properties=[
        "output.lobsterpy_data.cohp_plot_data",
        "output.lobsterpy_data_cation_anion.cohp_plot_data",
    ],
    load=True,
)

for key, cohp in result["output"]["lobsterpy_data"]["cohp_plot_data"]["data"].items():
    plotter = CohpPlotter()
    cohp_obj = Cohp.from_dict(cohp)
    plotter.add_cohp(key, cohp_obj)
    plotter.show()

for key, cohp in result["output"]["lobsterpy_data_cation_anion"]["cohp_plot_data"][
    "data"
].items():
    plotter = CohpPlotter()
    cohp_obj = Cohp.from_dict(cohp)
    plotter.add_cohp(key, cohp_obj)
    plotter.show()